
# Exploring Machine Learning through Sensor Data

This notebook is a focused, practical walk-through of an ML workflow for sensor data.  
It balances concise explanations with hands-on code and optional experiments so you can *learn by doing*.  

**Notebook structure (brief):**  
- Step 0 — Setting up the environment (quick, practical)  
- Step 1 — Getting to know the data (load & visualize)  
- Step 2 — Preparing the data (cleaning, smoothing, scaling)  
- Step 3 — Feature exploration (manual selection & insight)  
- Step 4 — Meet some learners (KNN, Logistic, SVM, Random Forest, optional LSTM)  
- Step 5 — Reflection and next steps

---

## Step 0 — Setting up your environment (concise)

1. Create and activate a virtual environment (recommended):
   - macOS / Linux:
     ```bash
     python -m venv ml_env
     source ml_env/bin/activate
     ```
   - Windows (PowerShell):
     ```powershell
     python -m venv ml_env
     .\ml_env\Scripts\Activate.ps1
     ```

2. Install required packages (run from folder containing `requirements.txt`):
```bash
pip install -r requirements.txt
```

3. Run the notebook:
```bash
jupyter notebook Exploring_ML_Sensor_Data.ipynb
```
or use the modern interface:
```bash
jupyter lab
```

**VS Code:** install the Python and Jupyter extensions, open the folder, pick the interpreter, then open this `.ipynb` and use the Run UI.

**Google Colab:** Upload this notebook to Colab and (optionally) run `!pip install -r requirements.txt` at the top of the runtime.

---

Run the next code cell (imports) and then the Environment Check cell that follows it.


In [ ]:

# Imports and basic configuration
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10,4)
SEED = 42
np.random.seed(SEED)


In [ ]:

# Environment check: run this cell to verify core packages
import sys, numpy as _np, pandas as _pd, sklearn as _sk, seaborn as _sns, matplotlib as _mpl
try:
    import tensorflow as _tf
    tf_version = _tf.__version__
except Exception:
    tf_version = 'NOT INSTALLED'
print('Python:', sys.version.splitlines()[0])
print('NumPy:', _np.__version__)
print('Pandas:', _pd.__version__)
print('scikit-learn:', _sk.__version__)
print('Seaborn:', _sns.__version__)
print('Matplotlib:', _mpl.__version__)
print('TensorFlow:', tf_version)
print('\nIf packages are missing, run: pip install -r requirements.txt')


## Step 1 — Getting to Know the Data

First, we load data. You can either use the **synthetic dataset** included here (useful to experiment quickly) or **load your own CSV files**.

The loader expects CSVs with typical sensor columns such as `Timestamp`, `Milliseconds`, `X`, `Y`, `Z` or variants. The loader will *not* preprocess your data — that comes later so you can explore raw behavior first.


In [ ]:

# --- Synthetic dataset (optional) ---
classes = ['walk','run','jump','pushup']
N = 500
frames = []
for cls in classes:
    if cls == 'walk':
        acc_base = [1.0, 1.2, 0.8]; gyr_base = [0.2, 0.1, 0.05]
    elif cls == 'run':
        acc_base = [3.0, 2.5, 3.5]; gyr_base = [0.6, 0.5, 0.4]
    elif cls == 'jump':
        acc_base = [5.0, 4.5, 6.0]; gyr_base = [0.8, 0.7, 0.6]
    else:
        acc_base = [0.5, 0.4, 0.6]; gyr_base = [0.15, 0.1, 0.05]
    acc = np.random.normal(loc=acc_base, scale=0.5, size=(N,3))
    gyr = np.random.normal(loc=gyr_base, scale=0.2, size=(N,3))
    dfc = pd.DataFrame(np.hstack([acc,gyr]), columns=['X','Y','Z','GX','GY','GZ'])
    dfc['label'] = cls
    dfc['time'] = pd.date_range('2021-01-01', periods=N, freq='100ms')
    dfc['source_file'] = f'synthetic_{cls}.csv'
    frames.append(dfc)
df_synthetic = pd.concat(frames, ignore_index=True)
print('Synthetic dataset ready — shape:', df_synthetic.shape)


### Data loading options

- **mode = 'synthetic'**: use the dataset created above.  
- **mode = 'auto'**: recursively search `data_root` for CSV files and load them.  
- **mode = 'manual'**: provide a `manual_files` list with explicit file paths.  

*Try it yourself:* If you have a folder of CSVs, set `mode='auto'` and `data_root` to the parent folder. The loader will infer labels from filenames (splitting at `_`) and create a `time` column when `Timestamp` is available.


In [ ]:

# Flexible loader (auto/manual/synthetic)
import glob, os

mode = 'synthetic'   # change to 'auto' or 'manual' as needed
data_root = './data' # used for 'auto' mode
manual_files = []    # list explicit paths when using manual mode, e.g. ['./data/run_1.csv']

def load_sensor_csv(path):
    try:
        df = pd.read_csv(path)
    except Exception as e:
        print('Failed to read', path, e); return pd.DataFrame()
    # unified time column
    if 'Timestamp' in df.columns and 'Milliseconds' in df.columns:
        df['time'] = pd.to_datetime(df['Timestamp']) + pd.to_timedelta(df['Milliseconds'], unit='ms')
    elif 'Timestamp' in df.columns:
        df['time'] = pd.to_datetime(df['Timestamp'])
    else:
        df['time'] = pd.RangeIndex(start=0, stop=len(df))
    # normalize column names
    colmap = {}
    for c in df.columns:
        cu = c.strip().upper()
        if cu in ['X','AX']: colmap[c] = 'X'
        if cu in ['Y','AY']: colmap[c] = 'Y'
        if cu in ['Z','AZ']: colmap[c] = 'Z'
        if cu in ['GX']: colmap[c] = 'GX'
        if cu in ['GY']: colmap[c] = 'GY'
        if cu in ['GZ']: colmap[c] = 'GZ'
    df = df.rename(columns=colmap)
    basename = os.path.basename(path)
    label = basename.split('_')[0]
    df['label'] = label
    df['source_file'] = basename
    return df

# load according to mode
if mode == 'synthetic':
    df_raw = df_synthetic.copy(); print('Using synthetic data (mode=synthetic)')
elif mode == 'auto':
    files = glob.glob(os.path.join(data_root, '**', '*.csv'), recursive=True)
    if not files:
        print('No CSVs found under', data_root, '- falling back to synthetic'); df_raw = df_synthetic.copy()
    else:
        print('Found', len(files), 'CSV files. Loading...'); dfs = [load_sensor_csv(p) for p in files]; df_raw = pd.concat(dfs, ignore_index=True)
else:
    if not manual_files:
        print('manual_files empty — using synthetic data'); df_raw = df_synthetic.copy()
    else:
        dfs = [load_sensor_csv(p) for p in manual_files]; df_raw = pd.concat(dfs, ignore_index=True); print('Loaded manual files.')

print('\nColumns available:', df_raw.columns.tolist())


> **Note:** The data you just loaded is *raw*. It may contain missing values, inconsistent sampling intervals, or sensor drift.  
> Don’t preprocess yet — explore first. What stands out when you scan the first few rows?



## Step 1b — Quick visual exploration

We’ll plot a short excerpt to get a feel for the signals. Visual inspection often reveals problems that statistics alone can hide.


In [ ]:

# Quick exploration: counts, summary, and a sample plot
if 'df_raw' not in globals():
    print('No data loaded yet — run the loader cell.')
else:
    df = df_raw.copy()
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'], errors='coerce')
    print('Samples per label:'); display(df['label'].value_counts())
    # pick a label to visualize
    lbl = df['label'].dropna().unique()[0] if 'label' in df.columns else None
    if lbl is not None:
        sample = df[df['label']==lbl].iloc[:1000]
        plot_cols = [c for c in ['X','Y','Z','GX','GY','GZ'] if c in sample.columns]
        if plot_cols:
            plt.figure(figsize=(12,4))
            for col in plot_cols:
                sns.lineplot(data=sample, x='time' if 'time' in sample.columns else sample.index, y=col, label=col)
            plt.title(f'Raw signals for label={lbl} (first samples)'); plt.show()
    display(df.describe().T)


**💡 Try it yourself:** Want to explore different synthetic behavior? Increase noise or change `N` in the synthetic cell and re-run loader + exploration.


In [ ]:

# Example (optional): increase noise for synthetic data
# noise_scale = 1.2
# acc = np.random.normal(loc=acc_base, scale=noise_scale, size=(N,3))
# (recreate df_synthetic by re-running the synthetic data cell with adjusted noise)



## Step 2 — Preparing the World for the Algorithm

Preprocessing helps convert messy signals into a consistent form models can reason about. It is not about making data “perfect” — it's about making important patterns clearer.


In [ ]:

# Preprocessing toggles (editable)
fill_missing = True
apply_smoothing = True
apply_standardization = True
resample_to_hz = None  # e.g., 20

if 'df_raw' not in globals():
    print('Run data loader first.')
else:
    df_processed = df_raw.copy()
    if fill_missing:
        df_processed = df_processed.fillna(method='ffill').fillna(method='bfill')
    if resample_to_hz and 'time' in df_processed.columns:
        out = []
        for lbl, g in df_processed.groupby('label'):
            g = g.set_index('time').sort_index()
            g_res = g.resample(f'{int(1000/resample_to_hz)}L').mean().interpolate()
            g_res['label'] = lbl
            out.append(g_res.reset_index())
        df_processed = pd.concat(out, ignore_index=True)
        print(f'Resampled to {resample_to_hz} Hz per label group')
    if apply_smoothing:
        cols = [c for c in ['X','Y','Z','GX','GY','GZ'] if c in df_processed.columns]
        df_processed[cols] = df_processed[cols].rolling(window=5, min_periods=1).mean()
    if apply_standardization:
        num_cols = df_processed.select_dtypes(include=['number']).columns.tolist()
        num_cols = [c for c in num_cols if c not in ['Milliseconds']]
        scaler = StandardScaler()
        df_processed[num_cols] = scaler.fit_transform(df_processed[num_cols])
    print('Preprocessing done — processed shape:', df_processed.shape)
    display(df_processed.head())


**💡 Try it yourself:** Toggle `apply_smoothing=False` or set `resample_to_hz=20`. Observe how plots and summary statistics change.


In [ ]:

# Example: alternative smoothing (paste to test)
# df_processed[['X','Y','Z']] = df_processed[['X','Y','Z']].rolling(window=15, min_periods=1).mean()
# display(df_processed[['X','Y','Z']].head())



## Step 3 — Feature Exploration (manual selection)

Choose features for deeper inspection. Look for redundancy, separability, or odd patterns that suggest sensor issues.


In [ ]:

# Feature selection (edit this list)
selected_features = ['X','Y','Z']
avail = df_processed.columns.tolist() if 'df_processed' in globals() else []
selected = [s for s in selected_features if s in avail]
if not selected:
    print('No valid selected features found in processed data. Update the selected_features list.')
else:
    print('Selected features:', selected)

In [ ]:

# Visualization for selected features
if 'df_processed' in globals() and selected:
    sns.pairplot(df_processed[selected].sample(min(1000, len(df_processed))), corner=True)
    plt.suptitle('Pairplot of selected features', y=1.02); plt.show()
    plt.figure(figsize=(6,5)); sns.heatmap(df_processed[selected].corr(), annot=True, cmap='coolwarm', center=0); plt.title('Correlation heatmap'); plt.show()



**💡 Try it yourself:** Add gyroscope axes (e.g., `'GX'`) to `selected_features`. Do those axes give new information or echo accelerometer axes?



## Step 4 — Meet some learners

Quick conceptual notes before running models:
- **KNN**: uses nearby instances — good when similar movements cluster close in feature space.  
- **Logistic (Softmax)**: simple linear decision boundaries, outputs class probabilities.  
- **SVM**: finds strong boundaries, can be sensitive to scaling.  
- **Random Forest**: ensembles of decision trees; useful for feature importance and robustness.  
- **LSTM (optional)**: captures temporal patterns when data is framed as short sequences.


In [ ]:

# Prepare numeric features and labels for modeling
if 'df_processed' not in globals():
    print('Run preprocessing first.')
else:
    num_cols = [c for c in df_processed.columns if c not in ['label','time','source_file','Milliseconds'] and pd.api.types.is_numeric_dtype(df_processed[c])]
    if not num_cols:
        raise ValueError('No numeric features found for modeling')
    print('Numeric columns used for modeling:', num_cols[:12])
    X = df_processed[num_cols].fillna(0).values
    y = df_processed['label'].values
    le = LabelEncoder(); y_enc = le.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=SEED)
    X_train_s = StandardScaler().fit_transform(X_train)
    X_test_s = StandardScaler().fit_transform(X_test)
    print('Prepared train/test datasets — shapes:', X_train_s.shape, X_test_s.shape)

In [ ]:

# K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_s, y_train)
knn_pred = knn.predict(X_test_s)
print('KNN accuracy:', accuracy_score(y_test, knn_pred))
print(classification_report(y_test, knn_pred))
ConfusionMatrixDisplay(confusion_matrix(y_test, knn_pred), display_labels=le.classes_).plot(cmap='Blues'); plt.title('KNN Confusion Matrix'); plt.show()

In [ ]:

# Logistic Regression (Softmax)
lr = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='saga')
lr.fit(X_train_s, y_train)
lr_pred = lr.predict(X_test_s)
print('Logistic Regression accuracy:', accuracy_score(y_test, lr_pred))
print(classification_report(y_test, lr_pred))
ConfusionMatrixDisplay(confusion_matrix(y_test, lr_pred), display_labels=le.classes_).plot(cmap='Oranges'); plt.title('Logistic Regression Confusion Matrix'); plt.show()

In [ ]:

# Support Vector Machine (SVM)
svm = SVC(kernel='rbf', C=1, gamma='scale')
svm.fit(X_train_s, y_train)
svm_pred = svm.predict(X_test_s)
print('SVM accuracy:', accuracy_score(y_test, svm_pred))
print(classification_report(y_test, svm_pred))
ConfusionMatrixDisplay(confusion_matrix(y_test, svm_pred), display_labels=le.classes_).plot(cmap='Purples'); plt.title('SVM Confusion Matrix'); plt.show()

In [ ]:

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=SEED)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print('Random Forest accuracy:', accuracy_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred))
importances = pd.Series(rf.feature_importances_, index=num_cols).sort_values(ascending=False)
plt.figure(figsize=(8,6)); importances[:12].plot(kind='barh'); plt.title('Top feature importances (Random Forest)'); plt.gca().invert_yaxis(); plt.show()

## Optional — small LSTM demo (conceptual)

This reshapes features into very short sequences if feasible. It's a toy demo to illustrate temporal models.

In [ ]:

# LSTM demo (runs only when reshape is feasible)
n_features = X_train_s.shape[1]
if n_features >= 3 and n_features % 3 == 0:
    timesteps = 3; step = n_features // timesteps
    X_seq = X_train_s.reshape((X_train_s.shape[0], timesteps, step))
    X_seq_test = X_test_s.reshape((X_test_s.shape[0], timesteps, step))
    y_seq = le.transform(y_train); y_seq_test = le.transform(y_test)
    model = Sequential([LSTM(32, input_shape=(timesteps, step)), Dense(len(le.classes_), activation='softmax')])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_seq, y_seq, epochs=4, batch_size=32, validation_split=0.1, verbose=2)
    loss, acc = model.evaluate(X_seq_test, y_seq_test, verbose=0)
    print('LSTM demo accuracy:', acc)
else:
    print('Skipping LSTM demo — reshape not suitable for current feature count')


## Step 5 — Reflection and next steps

**Questions to consider:**  
- Which preprocessing step changed the signals most — and why?  
- Which model seemed most robust to noise, scaling, or pocket variations?  
- If you were to deploy a model on-device, which preprocessing pipeline and model would you choose?

**Ideas for further exploration:**  
- Implement session-wise cross-validation to test generalization.  
- Try handcrafted features (e.g., standard deviation over windows) and compare with raw inputs.  
- Explore more advanced sequence models or on-device optimizations.
